In [133]:
import os
import cv2
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.optim.lr_scheduler import ReduceLROnPlateau

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [134]:
# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [135]:
class SatelliteDataset(Dataset):
    def print_transform(self):
        print(self.transform)
        
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 1]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image

        mask_rle = self.data.iloc[idx, 2]
        mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

In [136]:
# 3개의 다른 augmentation 방법을 정의합니다.
transform1 = A.Compose([
    A.RandomResizedCrop(224, 224),
    A.RandomShadow(p=0.5),
    A.HorizontalFlip(p=0.3),
    A.RandomRotate90(p=0.3),
    A.VerticalFlip(p=0.3),
    A.Normalize(),
    ToTensorV2()
])

transform2 = A.Compose([
    A.RandomResizedCrop(224, 224),
    A.RandomShadow(p=0.5),
    A.ColorJitter(p=0.3),
    A.Blur(blur_limit=3, p=0.3),
    A.Normalize(),
    ToTensorV2()
])

transform3 = A.Compose([
    A.RandomResizedCrop(224, 224),
    A.RandomShadow(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.GaussNoise(p=0.3),
    A.RandomGridShuffle(grid=(3, 3), p=0.3),
    A.Normalize(),
    ToTensorV2()
])

# 3개의 다른 augmentation 방법을 리스트로 결합합니다.
transforms_list = [transform1, transform2, transform3]

In [137]:
# U-Net의 기본 구성 요소인 Double Convolution Block을 정의합니다.
def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True)
    )

# 간단한 U-Net 모델 정의
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.dconv_down1 = double_conv(3, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = double_conv(128, 256)
        self.dconv_down4 = double_conv(256, 512)

        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)        

        self.dconv_up3 = double_conv(256 + 512, 256)
        self.dconv_up2 = double_conv(128 + 256, 128)
        self.dconv_up1 = double_conv(128 + 64, 64)

        self.conv_last = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)

        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)
        
        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)   

        x = self.dconv_down4(x)

        x = self.upsample(x)        
        x = torch.cat([x, conv3], dim=1)

        x = self.dconv_up3(x)
        x = self.upsample(x)        
        x = torch.cat([x, conv2], dim=1)       

        x = self.dconv_up2(x)
        x = self.upsample(x)        
        x = torch.cat([x, conv1], dim=1)   

        x = self.dconv_up1(x)

        out = self.conv_last(x)

        return out

In [138]:
# 모델 인스턴스 생성 및 디바이스 설정
model = UNet().to(device)

model.load_state_dict(torch.load('model_Unet_re.pt'))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# loss function과 optimizer 정의
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.000017)

# ReduceLROnPlateau 스케줄러 생성
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.9, patience=5, verbose=True)


# training loop
for epoch in range(50):
    # 매 에폭마다 다른 증강 기법을 선택합니다.
    current_transform = transforms_list[epoch % len(transforms_list)]
    
    # 학습 데이터셋 생성
    dataset = SatelliteDataset(csv_file='./train.csv', transform=current_transform)
    dataloader = DataLoader(dataset, batch_size=25, shuffle=True, num_workers=16)

    # 검증 데이터셋 생성
    val_dataset = SatelliteDataset(csv_file='./train.csv', transform=current_transform, infer=False)
    val_dataloader = DataLoader(val_dataset, batch_size=25, shuffle=False, num_workers=16)

    # dataset.print_transform()  # 현재 선택된 transform 출력
    
    model.train()
    epoch_loss = 0
    for images, masks in tqdm(dataloader):
        images = images.float().to(device)
        masks = masks.float().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks.unsqueeze(1))
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {epoch_loss/len(dataloader):.6f}')  # 학습 손실 출력 (소수점 아래 6자리까지)
    
    # ReduceLROnPlateau 스케줄러로 학습률 조정 (학습 데이터셋 손실 값을 기준으로)
    scheduler.step(epoch_loss)
    
    # 현재 학습률 출력
    current_lr = optimizer.param_groups[0]['lr']
    print(f'Current Learning Rate: {current_lr:.6f}')  # 학습률 출력 (소수점 아래 6자리까지)

100%|██████████| 286/286 [02:53<00:00,  1.65it/s]


Epoch 1, Loss: 0.045260
Current Learning Rate: 0.000017


100%|██████████| 286/286 [03:02<00:00,  1.57it/s]


Epoch 2, Loss: 0.039774
Current Learning Rate: 0.000017


100%|██████████| 286/286 [02:44<00:00,  1.74it/s]


...

Epoch 48, Loss: 0.038909
Epoch 00048: reducing learning rate of group 0 to 1.0038e-05.
Current Learning Rate: 0.000010


100%|██████████| 286/286 [02:29<00:00,  1.91it/s]


Epoch 49, Loss: 0.044777
Current Learning Rate: 0.000010


100%|██████████| 286/286 [02:29<00:00,  1.91it/s]

Epoch 50, Loss: 0.039203
Current Learning Rate: 0.000010


In [139]:
torch.save(model.state_dict(), "./model_Unet_re.pt")

In [140]:
def post_process(mask, min_pixel_threshold):
    # 객체 분할 마스크를 이진화
    mask_binary = (mask > 0).astype(np.uint8)

    # 객체별로 레이블링
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask_binary, connectivity=8)

    # 일정 픽셀 이하의 객체는 건물이 아니라고 판단하고 제거
    for label in range(1, num_labels):
        area = stats[label, cv2.CC_STAT_AREA]
        if area <= min_pixel_threshold:
            mask[labels == label] = 0

    return mask

In [141]:
transform1 = A.Compose(
    [   
        A.Resize(224, 224),
        A.Normalize(),
        ToTensorV2(),
    ]
)
test_dataset = SatelliteDataset(csv_file='./test.csv', transform=transform1, infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=25, shuffle=False, num_workers=16)


In [145]:
# 모폴로지 연산을 이용한 경계 보정 함수
def boundary_refinement(mask):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    refined_mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    return refined_mask

# 픽셀 그루핑 함수
def pixel_grouping(mask, min_pixel_threshold=20):
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=8)
    
    for label in range(1, num_labels):
        area = stats[label, cv2.CC_STAT_AREA]
        if area <= min_pixel_threshold:
            mask[labels == label] = 0
            
    return mask 

In [149]:
with torch.no_grad():
    #min_pixel_threshold = 20  # 일정 픽셀 이하의 객체를 건물이 아닌 것으로 판단
    model.eval()
    result = []
    for images in tqdm(test_dataloader):
        images = images.float().to(device)
        
        outputs = model(images)
        masks = torch.sigmoid(outputs).cpu().numpy()
        masks = np.squeeze(masks, axis=1)
        masks = (masks > 0.5).astype(np.uint8) 
    
        # 미분을 이용한 경계 보정 적용
        masks = [boundary_refinement(mask) for mask in masks]
        
        # 픽셀 그루핑 적용
        masks = [pixel_grouping(mask) for mask in masks]

        # 후처리 적용
        for i in range(len(images)):
            mask = masks[i]
            mask = post_process(mask, min_pixel_threshold)

            mask_rle = rle_encode(mask)
            if mask_rle == '':
                result.append(-1)
            else:
                result.append(mask_rle)

100%|██████████| 2426/2426 [08:58<00:00,  4.51it/s]


In [150]:
submit = pd.read_csv('./sample_submission.csv')
submit['mask_rle'] = result

In [151]:
submit.to_csv('./submitsum_re1.csv', index=False)